<a href="https://colab.research.google.com/github/DS-17-Projects-Tracking/NMPEDdata/blob/main/Mapped_Attendance_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [1]:
import pandas as pd
import numpy as np
import json
import requests
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [52]:
my_slider = widgets.IntSlider(description="Test")
display(my_slider)
print(f"Slider Model ID: {my_slider._model_id}")

IntSlider(value=0, description='Test')

Slider Model ID: 273484a18f3543e4bd4b7619db2d5ca8


## Edid pre-existing data

In [11]:
school_directory = "/content/drive/MyDrive/Robert Citek Tasks/Tableau   Looker Studio /Looker Studio/school_directory.csv"
data = pd.read_csv(school_directory)

In [12]:
MAIN_attendance = "/content/drive/MyDrive/Robert Citek Tasks/Tableau   Looker Studio /Looker Studio/MAIN_attendance_2019_2024.json"
df = pd.read_json(MAIN_attendance)

In [13]:
subgroups_to_drop = ("All,FRL,AnyNative,Foster,Military,Homeless,Multirace,DirectCert,SwD,AnyBlack").split(',')

In [14]:
df_dropped = df[~df["Subgroup"].isin(subgroups_to_drop)].copy()

In [15]:
df = df_dropped

In [16]:
df["Subgroup"].value_counts()

,count
Subgroup,
White,5075
EL,5075
Black,5075
Hispanic,5075
Asian,5075
Native,5075
Female,4204
Male,4204
Migrant,4204


In [17]:
data.head(5)

,ID,SchoolName,District,Level
0,1,albuquerque district,albuquerque,district
1,2,reserve district,reserve,district
2,3,quemado district,quemado,district
3,4,roswell district,roswell,district
4,5,hagerman district,hagerman,district


In [18]:
df.head(5)

,SchNumb,Subgroup,SY,Year,Number_Attending,Regular_Attendance_Rate
1,1,Female,2020-21,2021,36376.0,0.7559
2,1,Male,2020-21,2021,37369.0,0.7315
3,1,White,2020-21,2021,14455.0,0.8502
4,1,Black,2020-21,2021,1793.0,0.6771
5,1,Hispanic,2020-21,2021,49554.0,0.7129


In [19]:
school_map = data.set_index('ID')[['SchoolName', 'District','Level']].to_dict('index')

In [21]:
def map_school_info(schnumb):
    school_info = school_map.get(schnumb)
    if school_info:
        return school_info['SchoolName'], school_info['District'], school_info['Level']

In [22]:
df[['SchoolName_Mapped', 'District_Mapped', "Level Mapped"]] = df['SchNumb'].apply(lambda x: pd.Series(map_school_info(x)))

print("\nDataFrame 'df' with Mapped School Names and Districts:")
df.head()


DataFrame 'df' with Mapped School Names and Districts:


,SchNumb,Subgroup,SY,Year,Number_Attending,Regular_Attendance_Rate,SchoolName_Mapped,District_Mapped,Level Mapped
1,1,Female,2020-21,2021,36376.0,0.7559,albuquerque district,albuquerque,district
2,1,Male,2020-21,2021,37369.0,0.7315,albuquerque district,albuquerque,district
3,1,White,2020-21,2021,14455.0,0.8502,albuquerque district,albuquerque,district
4,1,Black,2020-21,2021,1793.0,0.6771,albuquerque district,albuquerque,district
5,1,Hispanic,2020-21,2021,49554.0,0.7129,albuquerque district,albuquerque,district


In [23]:
df_albuquerque = df[df['District_Mapped'] == 'albuquerque'].copy()
df_albuquerque.head()

,SchNumb,Subgroup,SY,Year,Number_Attending,Regular_Attendance_Rate,SchoolName_Mapped,District_Mapped,Level Mapped
1,1,Female,2020-21,2021,36376.0,0.7559,albuquerque district,albuquerque,district
2,1,Male,2020-21,2021,37369.0,0.7315,albuquerque district,albuquerque,district
3,1,White,2020-21,2021,14455.0,0.8502,albuquerque district,albuquerque,district
4,1,Black,2020-21,2021,1793.0,0.6771,albuquerque district,albuquerque,district
5,1,Hispanic,2020-21,2021,49554.0,0.7129,albuquerque district,albuquerque,district


## Funciton to plot graph per Schools

In [40]:
school_dropdown = widgets.Dropdown(options=sorted(df_albuquerque['SchoolName_Mapped'].unique()),description='Select School:',disabled=False)

In [41]:
def update_plot(selected_school):
    school_data = df_albuquerque[df_albuquerque['SchoolName_Mapped'] == selected_school]

    if not school_data.empty:
        fig, ax1 = plt.subplots(figsize=(12, 8))

        # Plot Attendance Rate
        sns.lineplot(x='Year', y='Regular_Attendance_Rate', hue='Subgroup', data=school_data, ax=ax1, marker='o')
        ax1.set_xlabel('Year')
        ax1.set_ylabel('Regular Attendance Rate', color='black')
        ax1.tick_params(axis='y', labelcolor='blue')
        ax1.set_title(f'Attendance Rate by Subgroup Over Time - {selected_school}\nLevel: {school_data["Level Mapped"].iloc[0]}, District: {school_data["District_Mapped"].iloc[0]}')
        ax1.grid(True)
        plt.legend(title='Subgroup')

    else:
        print(f"No data available for {selected_school}")


In [42]:
output = widgets.interactive_output(update_plot, {'selected_school': school_dropdown})

# Display the dropdown and the output plot
display(school_dropdown, output)

Dropdown(description='Select School:', options=('a. montoya elementary', 'abq charter academy', 'ace leadershi…

Output()

In [28]:
df_albuquerque.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7158 entries, 1 to 75490
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   SchNumb                  7158 non-null   int64  
 1   Subgroup                 7158 non-null   object 
 2   SY                       7158 non-null   object 
 3   Year                     7158 non-null   int64  
 4   Number_Attending         4364 non-null   float64
 5   Regular_Attendance_Rate  4358 non-null   float64
 6   SchoolName_Mapped        7158 non-null   object 
 7   District_Mapped          7158 non-null   object 
 8   Level Mapped             7158 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 559.2+ KB


In [29]:
df_albuquerque["Subgroup"].value_counts()

,count
Subgroup,
White,852
EL,852
Black,852
Hispanic,852
Asian,852
Native,852
Female,682
Male,682
Migrant,682


In [30]:
# df.to_csv('Mapped_Attendance_NMPED.csv', index=False)

## Some Viz created using Looker. The following it's how we take the embeded url and use it on our Jupyter notebooks


In [50]:
#Regular attendance per school selection

In [48]:
IFrame(src="https://lookerstudio.google.com/embed/reporting/c43f6ed2-c109-4a5c-8967-b6b901b2446b/page/CTGGF", width=800, height=500)

In [ ]:
# Reg attendance vs SY

In [49]:
IFrame(src="https://lookerstudio.google.com/embed/reporting/4d7065a9-661f-4a78-b969-ce5b06e4c28e/page/haGGF", width=800, height=500)